In [1]:
import os, re, time, sys
import pandas as pd
from  geopy.geocoders import Nominatim

In [33]:
def text2int(textnum, numwords={}):
    if not numwords:
      units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
      ]

      tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

      scales = ["hundred", "thousand", "million", "billion", "trillion"]

      numwords["and"] = (1, 0)
      for idx, word in enumerate(units):    numwords[word] = (1, idx)
      for idx, word in enumerate(tens):     numwords[word] = (1, idx * 10)
      for idx, word in enumerate(scales):   numwords[word] = (10 ** (idx * 3 or 2), 0)

    current = result = 0
    for word in textnum.split():
        if word not in numwords:
            break
#           raise Exception("Illegal word: " + word)
            

        scale, increment = numwords[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0

    return result + current

In [174]:
def locFromText(text):
    tofilter = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen","strike","strikes","near","destroyed","the","engaged"
      ]
    t = re.search(r'Near (.*?)(?:[,;]|(?!.*\b(?:one|two|three|four|five|six|seven|eight|nine|ten)\b)){1,3}', text)
    if t:
        red=' '.join(t.group(1).split()[:5])
        red = red.replace('and','|')
        red = red.replace('-',' ')
        red = red.replace('ar','al')
        red = red.lower()
        if len(red) < 2:
            return None
        else:
            return " ".join([w for w in red.split() if not w in tofilter])
    else:
        return None

In [176]:
df = pd.read_feather('allreports.feather')
df.shape

(9723, 4)

In [177]:
df["loc"] = df.apply(lambda x: locFromText(x.text), axis=1)
print(df["loc"].unique())

['al shadaddi' 'ar raqqah' 'tabqah' 'al qaim' 'haditha' 'mosul' 'tal afal'
 'dayr az zawr' None 'huwijah' 'rawah' 'abu kamal' 'raqqah' 'fallujah'
 'hawijah' 'al hasakah' 'kobani' 'bayji' 'sinjal' 'mal’a' 'palmyra'
 'sultan abdallah' 'tal abyad' 'baghdadi' 'al huwayjah' 'makhmur' 'ramadi'
 'aleppo' 'al asad' 'kirkuk' 'manbij' 'hit' 'kisik' 'qayyalah'
 'al baghdadi' 'ayn isa' 'habbaniyah' 'albu hayat' 'at tanf' 'rutbah'
 'tuz' 'hawayjah' 'bashir' 'al hawl' 'al hawijah' 'tikrit' 'ar rutbah'
 'samalra' 'mahkmur' 'washiyah' 'habbaniya' 'taji' 'hajin' 'sulayman bek'
 'tamakh' 'al kisik' 'ash shaddadi' 'al huwajah'
 'anbal desert | wadi ashai' 'makmur mountains' 'makmur' 'lake hamrin'
 'al huwayja' 'hasakah' 'day az zawr' 'al shaddadi' 'hamrin mountains'
 'huwayjah' 'dayr ar zawr' 'dayz az zawr' 'tallafal' 'al bashir'
 'hawijah pocket' 'baghdad' 'waleed' 'al soor' 'hit o' 'lake qadisiyyah'
 'jazeerah desert' 'iblib' 'al bab' 'tal hamis' 'idlib' 'dawr az zawr'
 'tabqa' 'al hawayjah' 'jalula' '

In [161]:
x = df["loc"].unique()
loc=pd.DataFrame.from_records(x)

TypeError: 'NoneType' object is not iterable

In [70]:
# Worried about dead values?
# df[df.text.map(len) > 10]

df.apply(lambda x : x["text"][x["text"].find("Near")],axis=1).unique()

array(['N', ' ', 'q'], dtype=object)

In [72]:
# Get Number of Strikes
df["strikes"] = df.apply(lambda x: x["text"].split(' strike',1)[0].split(', ')[-1],axis=1)
df["strikes"] = df.apply(lambda x: x.strikes if x.strikes.isnumeric() else text2int(x.strikes), axis=1)

<bound method NDFrame.head of 0        1
1        2
2       13
3        1
4        1
        ..
9718     0
9719     0
9720     0
9721     0
9722     0
Name: strikes, Length: 9723, dtype: object>

In [60]:
loc.head

AttributeError: 'Location' object has no attribute 'head'

In [53]:
geolocator = Nominatim()
areas["loc"] = areas.apply(lambda x: geolocator.geocode(x["loc"]+','+ "SY"), axis=1 )
df.head

/Users/andrewlb/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


AttributeError: 'numpy.ndarray' object has no attribute 'apply'